Adding Necessary Libraries

In [2]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

Using TensorFlow backend.


specify the Glove Embedding file path

GLOVE_DIR='./GLOVE.6B/'

Defining Parameters

In [4]:
# make the max word length to be constant
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
# the percentage of train test split to be applied
VALIDATION_SPLIT = 0.20
# the dimension of vectors to be used
EMBEDDING_DIM = 100
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
# dropout probability
drop = 0.5
batch_size = 30
epochs =30

specifing the Dtaset path

In [5]:
TEXT_DATA_DIR = r'./20newsgroup/20-News-Group'

Read the text and preparing the Class Labels

In [6]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)
print(labels_index)

print('Found %s texts.' % len(texts))
print(labels)

{'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2, 'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5, 'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9, 'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13, 'sci.space': 14, 'soc.religion.christian': 15, 'talk.politics.guns': 16, 'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}
Found 19997 texts.
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Preparing word sequences with uniform length and Preparing the word_index

In [8]:
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(texts))
sequences =  tokenizer.texts_to_sequences(texts)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print(len(labels))

None
19997
unique words : 174074
Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)
19997


 split the data into a training set and a validation set

In [9]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [10]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Preparing Word Embedding matrix from Glove vectors

In [11]:
embeddings_index = {}
FILE=os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = open(FILE,encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Preparing word embedding matrix for given dataset

In [12]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [13]:
print(embedding_matrix.shape)

(174075, 100)


feeding input data to cnn model.

In [14]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                               input_length=MAX_SEQUENCE_LENGTH,
                             trainable=False)

Configuring CNN Model

In [15]:
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding = embedding_layer(inputs)

print(embedding.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedding)
print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] +
                                 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)


concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=20, activation='softmax')(dropout)
print(output)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(None, 1000, 100)
(None, 1000, 100, 1)
Tensor("dense_1/Softmax:0", shape=(None, 20), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    17407500    input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1000, 100, 1) 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 998, 1, 512)  154112      reshape_1[0][0]           

Training and Testing CNN Model

In [16]:
print("Traning Model...")
model.fit(x_train, y_train, batch_size=50, epochs=50 , verbose=1, callbacks=[checkpoint])
score = model.evaluate(x_test, y_test, verbose=0)  
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Traning Model...
Epoch 1/50
15998/15998 [==============================] - 1357s 85ms/step - loss: 3.6286 - accuracy: 0.0968
Epoch 2/50


C:\Users\Chindukuri\.conda\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


15998/15998 [==============================] - 1450s 91ms/step - loss: 2.6199 - accuracy: 0.2367
Epoch 3/50
15998/15998 [==============================] - 1471s 92ms/step - loss: 2.0496 - accuracy: 0.3701
Epoch 4/50
15998/15998 [==============================] - 1427s 89ms/step - loss: 1.6653 - accuracy: 0.4870
Epoch 5/50
15998/15998 [==============================] - 1425s 89ms/step - loss: 1.4238 - accuracy: 0.5641
Epoch 6/50
15998/15998 [==============================] - 1421s 89ms/step - loss: 1.2453 - accuracy: 0.6225
Epoch 7/50
15998/15998 [==============================] - 1296s 81ms/step - loss: 1.1064 - accuracy: 0.6646
Epoch 8/50
15998/15998 [==============================] - 1270s 79ms/step - loss: 1.0183 - accuracy: 0.6943
Epoch 9/50
15998/15998 [==============================] - 1252s 78ms/step - loss: 0.9385 - accuracy: 0.7187
Epoch 10/50
15998/15998 [==============================] - 1253s 78ms/step - loss: 0.8577 - accuracy: 0.7470
Epoch 11/50
15998/15998 [=============

saving the cnn model by saving best weihts into h5 file.

In [20]:
from keras.models import model_from_json
from keras.models import load_model
from keras.models import Model
model_json = model.to_json()
with open("CNN_model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("CNN_model.h5")
json_file = open('CNN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

Loading the CNN Model to predict the class labels for test data

In [142]:
from keras.models import model_from_json
from keras.models import load_model
from keras.models import Model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("best_model.h5")
print("Loaded model from disk")

Loaded model from disk


Specify the test data path

In [159]:
test_data='./test_data'

Read the data from all test files and preparing labels.

In [160]:

import os
import sys
Test_texts = []  # list of text samples
Test_labels_index = {}  # dictionary mapping label name to numeric id
Test_labels = []  # list of label ids
Test_files=[]
for name in sorted(os.listdir(test_data)):
    #Test_files.append(name)
    path = os.path.join(test_data, name)
    #Test_files.append(path)
    if os.path.isdir(path):
        Test_label_id = len(Test_labels_index) 
        Test_labels_index[name] = Test_label_id
        for fname in sorted(os.listdir(path)):
            #Test_files.append(fname)
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                #Test_files.append(fpath)
               # print(fpath)
                if sys.version_info < (3,):
                    f = open(fpath)
                else: 
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                Test_texts.append(t)
                f.close()
                Test_labels.append(Test_label_id)
print(Test_labels_index)

print('Found %s texts.' % len(Test_texts))
#print(Test_labels)

{'alt.atheism': 0, 'comp.graphics': 1, 'comp.os.ms-windows.misc': 2, 'comp.sys.ibm.pc.hardware': 3, 'comp.sys.mac.hardware': 4, 'comp.windows.x': 5, 'misc.forsale': 6, 'rec.autos': 7, 'rec.motorcycles': 8, 'rec.sport.baseball': 9, 'rec.sport.hockey': 10, 'sci.crypt': 11, 'sci.electronics': 12, 'sci.med': 13, 'sci.space': 14, 'soc.religion.christian': 15, 'talk.politics.guns': 16, 'talk.politics.mideast': 17, 'talk.politics.misc': 18, 'talk.religion.misc': 19}
Found 2681 texts.


In [161]:
test_lables_set=Test_labels_index

In [162]:
labels1=list(test_lables_set.keys())

In [163]:
labels1

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [164]:
print(len(Test_labels))

2681


In [165]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
import numpy as np
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
tokenizer  = Tokenizer(num_words = MAX_WORDS)
tokenizer.fit_on_texts(Test_texts)
sequences =  tokenizer.texts_to_sequences(Test_texts)

Test_word_index = tokenizer.word_index
print("unique words : {}".format(len(Test_word_index)))

Test_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Test_labels1 = to_categorical(np.asarray(Test_labels))

unique words : 41129


In [166]:
print(len(Test_labels))

2681


In [168]:
print(len(Test_data))

2681


In [169]:
from keras.models import model_from_json
from keras.models import load_model
from keras.models import Model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("best_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [170]:
i=0
j=0
for x_t in Test_data:
    #print(x_t)
    prediction = loaded_model.predict(np.array([x_t]))
    #print(prediction)
    predicted_label = labels1[np.argmax(prediction[0])]
    print( "File ->", labels1[np.argmax(Test_labels1[i])],"Predicted label: " + predicted_label)
    if(labels1[np.argmax(Test_labels1[i])]==predicted_label):
        j=j+1
    i += 1
print(j)

File -> alt.atheism Predicted label: talk.religion.misc
File -> alt.atheism Predicted label: comp.graphics
File -> alt.atheism Predicted label: talk.politics.misc
File -> alt.atheism Predicted label: sci.med
File -> alt.atheism Predicted label: sci.crypt
File -> alt.atheism Predicted label: alt.atheism
File -> alt.atheism Predicted label: sci.med
File -> alt.atheism Predicted label: talk.religion.misc
File -> alt.atheism Predicted label: alt.atheism
File -> alt.atheism Predicted label: sci.crypt
File -> alt.atheism Predicted label: talk.politics.misc
File -> alt.atheism Predicted label: comp.os.ms-windows.misc
File -> alt.atheism Predicted label: alt.atheism
File -> alt.atheism Predicted label: talk.religion.misc
File -> alt.atheism Predicted label: talk.politics.misc
File -> alt.atheism Predicted label: comp.os.ms-windows.misc
File -> alt.atheism Predicted label: talk.religion.misc
File -> alt.atheism Predicted label: sci.crypt
File -> alt.atheism Predicted label: talk.politics.mideas

File -> comp.graphics Predicted label: sci.space
File -> comp.graphics Predicted label: talk.religion.misc
File -> comp.graphics Predicted label: rec.autos
File -> comp.graphics Predicted label: sci.crypt
File -> comp.graphics Predicted label: rec.motorcycles
File -> comp.graphics Predicted label: talk.politics.misc
File -> comp.graphics Predicted label: talk.religion.misc
File -> comp.graphics Predicted label: talk.politics.mideast
File -> comp.graphics Predicted label: talk.politics.guns
File -> comp.graphics Predicted label: talk.politics.misc
File -> comp.graphics Predicted label: talk.politics.misc
File -> comp.graphics Predicted label: comp.sys.mac.hardware
File -> comp.graphics Predicted label: comp.windows.x
File -> comp.graphics Predicted label: alt.atheism
File -> comp.graphics Predicted label: soc.religion.christian
File -> comp.graphics Predicted label: alt.atheism
File -> comp.graphics Predicted label: alt.atheism
File -> comp.graphics Predicted label: talk.politics.mideas

File -> comp.graphics Predicted label: talk.politics.misc
File -> comp.graphics Predicted label: talk.politics.mideast
File -> comp.graphics Predicted label: talk.politics.mideast
File -> comp.graphics Predicted label: alt.atheism
File -> comp.graphics Predicted label: comp.graphics
File -> comp.graphics Predicted label: sci.electronics
File -> comp.graphics Predicted label: comp.os.ms-windows.misc
File -> comp.os.ms-windows.misc Predicted label: comp.sys.mac.hardware
File -> comp.os.ms-windows.misc Predicted label: comp.graphics
File -> comp.os.ms-windows.misc Predicted label: rec.sport.baseball
File -> comp.os.ms-windows.misc Predicted label: talk.politics.misc
File -> comp.os.ms-windows.misc Predicted label: talk.religion.misc
File -> comp.os.ms-windows.misc Predicted label: alt.atheism
File -> comp.os.ms-windows.misc Predicted label: misc.forsale
File -> comp.os.ms-windows.misc Predicted label: comp.windows.x
File -> comp.os.ms-windows.misc Predicted label: alt.atheism
File -> comp

File -> rec.autos Predicted label: comp.sys.mac.hardware
File -> rec.autos Predicted label: talk.politics.misc
File -> rec.autos Predicted label: talk.religion.misc
File -> rec.autos Predicted label: comp.sys.mac.hardware
File -> rec.autos Predicted label: rec.sport.hockey
File -> rec.autos Predicted label: comp.sys.mac.hardware
File -> rec.autos Predicted label: sci.med
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: talk.politics.guns
File -> rec.autos Predicted label: misc.forsale
File -> rec.autos Predicted label: misc.forsale
File -> rec.autos Predicted label: talk.politics.mideast
File -> rec.autos Predicted label: rec.sport.hockey
File -> rec.autos Predicted label: talk.politics.misc
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: talk.politics.misc
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted 

File -> rec.autos Predicted label: comp.windows.x
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: talk.politics.guns
File -> rec.autos Predicted label: talk.politics.misc
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: comp.graphics
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: talk.politics.guns
File -> rec.autos Predicted label: comp.graphics
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: talk.politics.mideast
File -> rec.autos Predicted label: talk.politics.misc
File -> rec.autos Predicted label: alt.atheism
File -> rec.autos Predicted label: sci.med
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: sci.space
File -> rec.autos Predicted label: talk.religion.misc
File -> rec.autos Predicted label: comp.windows.x
File -> rec.autos P

...

File -> sci.crypt Predicted label: sci.space
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: talk.politics.misc
File -> sci.crypt Predicted label: rec.sport.hockey
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: rec.sport.hockey
File -> sci.crypt Predicted label: talk.politics.mideast
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: talk.politics.mideast
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: soc.religion.christian
File -> sci.crypt Predicted label: comp.sys.ibm.pc.hardware
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.c

File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: sci.med
File -> sci.crypt Predicted label: rec.autos
File -> sci.crypt Predicted label: talk.politics.guns
File -> sci.crypt Predicted label: talk.politics.mideast
File -> sci.crypt Predicted label: soc.religion.christian
File -> sci.crypt Predicted label: comp.windows.x
File -> sci.crypt Predicted label: comp.os.ms-windows.misc
File -> sci.crypt Predicted label: talk.politics.misc
File -> sci.crypt Predicted label: sci.electronics
File -> sci.crypt Predicted label: talk.politics.misc
File -> sci.crypt Predicted label: talk.politics.misc
File -> sci.crypt Predicted label: talk.politics.guns
File -> sci.crypt Predicted label: sci.space
File -> sci.crypt Predicted label: comp.windows.x
File -> sci.crypt Predicted label: talk.politics.guns
File -> sci.crypt Predicted label: comp.sys.mac.hardware
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: sci.space
File -> sci.cry

File -> sci.crypt Predicted label: sci.space
File -> sci.crypt Predicted label: sci.crypt
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: talk.religion.misc
File -> sci.crypt Predicted label: rec.sport.hockey
File -> sci.crypt Predicted label: talk.politics.mideast
File -> sci.crypt Predicted label: sci.med
File -> sci.crypt Predicted label: rec.sport.hockey
File -> sci.crypt Predicted label: sci.space
File -> sci.crypt Predicted label: soc.religion.christian
File -> sci.crypt Predicted label: rec.sport.baseball
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: alt.atheism
File -> sci.crypt Predicted label: sci.space
File -> sci.crypt Predicted label: talk.politics.guns
File -> sci.crypt Pr

File -> sci.space Predicted label: alt.atheism
File -> sci.space Predicted label: alt.atheism
File -> sci.space Predicted label: talk.politics.guns
File -> sci.space Predicted label: talk.politics.mideast
File -> sci.space Predicted label: alt.atheism
File -> sci.space Predicted label: alt.atheism
File -> sci.space Predicted label: talk.politics.mideast
File -> sci.space Predicted label: alt.atheism
File -> sci.space Predicted label: talk.politics.misc
File -> sci.space Predicted label: sci.space
File -> sci.space Predicted label: talk.politics.misc
File -> sci.space Predicted label: talk.politics.mideast
File -> sci.space Predicted label: talk.politics.misc
File -> sci.space Predicted label: comp.sys.ibm.pc.hardware
File -> sci.space Predicted label: comp.graphics
File -> sci.space Predicted label: comp.graphics
File -> sci.space Predicted label: comp.sys.ibm.pc.hardware
File -> sci.space Predicted label: soc.religion.christian
File -> sci.space Predicted label: talk.politics.misc
Fil

File -> talk.religion.misc Predicted label: rec.autos
File -> talk.religion.misc Predicted label: talk.politics.misc
File -> talk.religion.misc Predicted label: talk.religion.misc
File -> talk.religion.misc Predicted label: talk.religion.misc
File -> talk.religion.misc Predicted label: talk.politics.mideast
File -> talk.religion.misc Predicted label: comp.os.ms-windows.misc
File -> talk.religion.misc Predicted label: comp.os.ms-windows.misc
File -> talk.religion.misc Predicted label: sci.space
File -> talk.religion.misc Predicted label: soc.religion.christian
File -> talk.religion.misc Predicted label: talk.politics.guns
113
